In [ ]:
# contours for 2 eyes
import cv2
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import numpy as np
import imutils
import time

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Constants for eye blink detection
EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 3

# Initialize variables for eye blink detection
COUNTER = 0
TOTAL = 0

# Open a connection to the camera (0 represents the default camera)
vs = VideoStream(src=0).start()
time.sleep(1.0)

while True:
    # Capture frame-by-frame
    frame = vs.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Draw the blue rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Extract the region of interest for eye blink detection
        roi_gray = gray[y:y + h, x:x + w]

        # Extract the left and right eye coordinates
        leftEye = roi_gray[5:11, 0:w]
        rightEye = roi_gray[5:11, w - 6:w]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        # Draw contours for the eyes inside the detected face region
        leftEyeHull = cv2.convexHull(np.array([(50, 100), (50, 70), (w-120, 100), (w-120, 70)]))
        rightEyeHull = cv2.convexHull(np.array([(50, 100), (50, 70), (w-120, 100), (w-120, 70)]))

        # Draw green contours inside the blue rectangles for the eyes
        cv2.drawContours(frame[y:y + h, x:x + w], [leftEyeHull], -1, (0, 255, 0), 2)
        cv2.drawContours(frame[y:y + h, x:x + w], [rightEyeHull], -1, (0, 255, 0), 2)

        leftEyeHull2 = cv2.convexHull(np.array([(130, 100), (130, 70), (w-40, 100), (w-40, 70)]))
        rightEyeHull2 = cv2.convexHull(np.array([(130, 100), (130, 70), (w-40, 100), (w-40, 70)]))

        # Draw green contours inside the blue rectangles for the eyes
        cv2.drawContours(frame[y:y + h, x:x + w], [leftEyeHull2], -1, (0, 255, 0), 2)
        cv2.drawContours(frame[y:y + h, x:x + w], [rightEyeHull2], -1, (0, 255, 0), 2)

        # Convert the region of interest (ROI) to HSV
        roi_color = frame[y:y + h, x:x + w]
        roi_hsv = cv2.cvtColor(roi_color, cv2.COLOR_BGR2HSV)

        # Define a mask to identify bright colors (you can adjust the values based on your specific requirements)
        lower_bright = np.array([0, 0, 200])
        upper_bright = np.array([180, 255, 255])
        mask_bright = cv2.inRange(roi_hsv, lower_bright, upper_bright)

        # Find contours in the mask
        contours_bright, _ = cv2.findContours(mask_bright, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Check if there are any bright color contours inside the eyes
        if len(contours_bright) > 0:
            status = "Awake"
        else:
            status = "Closed"

        # Display the status on the frame
        cv2.putText(frame, f"Status: {status}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Check for blinks
        if ear < EYE_AR_THRESH:
            COUNTER += 1
        else:
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                TOTAL += 1
            COUNTER = 0

    # Display the resulting frame
    cv2.imshow('Face and Eye Detection', frame)

    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the camera and close the window
vs.stop()
cv2.destroyAllWindows()
